Different nets:
- few first only on random agent
- 3, 4 -> based on every saved individual equally
- 5 -> on data3 (100.000 random, rest from 950 with noise)
- 6 -> on data4 with smaller batches to forward

In [0]:
import numpy as np
import torch
from torch import nn
import torch.nn.functional as F
from copy import deepcopy

import numpy as np
from tqdm import trange
import glob
import io
import base64
from IPython.display import HTML

from IPython import display as ipythondisplay
import pickle

In [0]:
CUDA = torch.cuda.is_available()

def to_np(x):
    return x.detach().cpu().numpy()

def to_tensor(x, requires_grad=False):
    x = torch.from_numpy(x)
    if CUDA:
        x = x.cuda()
    
    if requires_grad:
        return x.clone().contiguous().detach().requires_grad_(True)
    else:
        return x.clone().contiguous().detach()

    
class Network(nn.Module):
    def __init__(self):
        super(Network, self).__init__()
        self.layers = nn.Sequential(
            nn.Linear(119, 500),
            nn.Tanh(),
            nn.Linear(500, 500),
            nn.Tanh(),
            nn.Linear(500, 500),
            nn.Tanh(),
            nn.Linear(500, 1)
        )
        self.loss = nn.MSELoss()
        
        
    def forward(self, X):
        X = X.view(X.size(0), -1)
        return self.layers.forward(X)

In [0]:
# data = pickle.load(open('drive/My Drive/project_evo/data.pkl', 'rb'))

In [0]:
def train(net=Network()):
    if CUDA:
        net = net.cuda()

    optimizer = torch.optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

    for _ in trange(1000, position=0, leave=True):
        for bch in range(10):
            d = pickle.load(open(f'drive/My Drive/project_evo/data4/b{bch}.pkl', 'rb'))
            X, Y = d['X'], d['Y']
            if CUDA:
                X, Y = X.cuda(), Y.cuda()

            for i in range(0, len(X), 200):
                preds = net(X[i : i+200])
                optimizer.zero_grad()
                loss = net.loss(Y[i : 200+i], preds)
                loss.backward()
                optimizer.step()
    return net

In [0]:
net = train()

100%|██████████| 1000/1000 [4:32:59<00:00, 16.38s/it]


In [0]:
L = []
with torch.no_grad():
    for bch in trange(100, position=0, leave=True):
        d = pickle.load(open(f'drive/My Drive/project_evo/data/b{bch}.pkl', 'rb'))
        X, Y = d['X'][:500], d['Y'][:500]
        if CUDA:
            X, Y = X.cuda(), Y.cuda()

        preds = net(X)
        loss = net.loss(Y, preds)
        L.append(loss.item())
print(np.mean(L))

In [0]:
pickle.dump(net, open('drive/My Drive/project_evo/reward_pred/net6.pkl', 'w+b'), pickle.HIGHEST_PROTOCOL)
pickle.dump(net.to(torch.device('cpu')), open('drive/My Drive/project_evo/reward_pred/net6_cpu.pkl', 'w+b'), pickle.HIGHEST_PROTOCOL)

/usr/local/lib/python3.6/dist-packages/torch/storage.py:34: FutureWarning: pickle support for Storage will be removed in 1.5. Use `torch.save` instead
  warnings.warn("pickle support for Storage will be removed in 1.5. Use `torch.save` instead", FutureWarning)


In [0]:
# pickle.dump(net, open('drive/My Drive/project_evo/reward_pred/net2_cpu.pkl', 'w+b'))
net = pickle.load(open('drive/My Drive/project_evo/reward_pred/net2.pkl', 'rb'))

In [0]:
pickle.dump(net.to(torch.device('cpu')), open('drive/My Drive/project_evo/reward_pred/net2_cpu.pkl', 'w+b'), pickle.HIGHEST_PROTOCOL)

/usr/local/lib/python3.6/dist-packages/torch/storage.py:34: FutureWarning: pickle support for Storage will be removed in 1.5. Use `torch.save` instead
  warnings.warn("pickle support for Storage will be removed in 1.5. Use `torch.save` instead", FutureWarning)
